In [1]:
import tensorflow as tf
import keras as kr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from functools import reduce

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Input, MaxPooling2D, AveragePooling2D

%matplotlib inline

Using TensorFlow backend.


In [2]:
NUMBER_SIZE = 15
THRESHOLD_COUNT = 5

BATCH_NUMBER_COUNT_50 = 50
SINGLE_SAMPLE_SIZE_50 = BATCH_NUMBER_COUNT_50 * NUMBER_SIZE

CLASSES = { "none": 0, "attack": 1 }

In [3]:
def create_single_sample_from_array(numbers):
  # Treat the list of phone numbers as a single huge list digits
  # pretty much like greyscale images, etc.
  digit_str = ''.join([str(i) for i in numbers])
  # Now create an image-like array
  return list(digit_str)

def create_single_sample_from_dataframe(dataframe_row):
  return create_single_sample_from_array(np.array(dataframe_row))

def combine_dfs(dfs, outcome_values=None, shuffle=True):
  # Combine data with sequence with outcome
  df_list = []
  for idx, df in enumerate(dfs):
    data_arr = []
    for index, series in df.iterrows():
      series_value = [v for _, v in series.iteritems()]
      combined_series = create_single_sample_from_array(series_value)
      data_arr.append(combined_series)
    
    df_arr_row_count = len(data_arr)
    
    outcome_value = outcome_values[idx] if outcome_values else idx
    data_outcome_tuples = zip(data_arr, [outcome_value] * df_arr_row_count)

    # DEBUG
    print("outcome_value:%s of all possible outcome_values:%s" % (outcome_value, outcome_values))

    df_list += data_outcome_tuples   
    
  # Merge data with sequence and no sequence
  dataframe = pd.DataFrame(df_list, columns=["sample","outcome"])
  # Shuffle (or rather randomly select samples) but 1.0 means all
  df_random = dataframe.sample(frac=1) if shuffle else dataframe
  return df_random

def extract_sample_and_outcome(df, sample_col_name='sample', outcome_col_name='outcome'):
  X = [i for i in df[sample_col_name]]
  Y = [i for i in df[outcome_col_name]]
  return (X, Y)

def load_data(filename, expected_shape_tuple):
  data = pd.read_csv(filename, header=None)
  print("data file:%s, data.shape (should be %s): %s" % (filename, expected_shape_tuple, data.shape))
  if data.shape != expected_shape_tuple:
    raise ValueError("data.shape:%s does not match excpected shape:%s" % (data.shape, expected_shape_tuple))
  return data
    
def prepare_data(filenames, expected_shape_tuples, outcome_values=None, shuffle=True):
  dfs = []
  for idx, filename in enumerate(filenames):
    dfs.append(load_data(filename, expected_shape_tuples[idx]))

  df_all = combine_dfs(dfs, outcome_values, shuffle)
  if (df_all.shape[0] != reduce((lambda m, i: m + i.shape[0]), dfs,0)):
    raise ValueError("There is a problem with combine_dfs. df_all.shape:%s does not match the sum of df_seq:%s and df_no_seq:%s" % (df_all.shape, df_seq.shape, df_no_seq.shape))
  return df_all

def prepare_train_data(dataframe, single_sample_size):
  df_row_count = dataframe.shape[0]
  (X, Y) = extract_sample_and_outcome(dataframe)
  X_train = np.array(X).reshape(df_row_count, 1, single_sample_size, 1)
  Y_train = np.array(Y).reshape(df_row_count)

  print("type(X):%s, len(X):%d, len(X[0]):%d" % (type(X), len(X), len(X[0])))
  print("df_row_count: %d" % df_row_count)
  print("single_sample_size_bit:%d" % single_sample_size)
  print("X_train.shape:%s, Y_train.shape:%s" % (X_train.shape, Y_train.shape))
  print("\n")

  print("type(Y[0]):%s, Y[0]:%s" % (type(Y[0]), Y[0]))

  return (X_train, Y_train)

In [5]:
# Load data from files into dataframe

df_random_2040_2040_combo_sample_number_50 = prepare_data(['data_no_sequence_2040_sample_number_50.csv','data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_2040_sample_number_50.csv'], [(2040,50), (2040,50)], [CLASSES["none"],CLASSES["attack"]]) 

df_random_10200_10200_combo_sample_number_50 = prepare_data(['data_no_sequence_10200_sample_number_50.csv','data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_10200_sample_number_50.csv'], [(10200,50), (10200,50)], [CLASSES["none"],CLASSES["attack"]]) 

data file:data_no_sequence_2040_sample_number_50.csv, data.shape (should be (2040, 50)): (2040, 50)
data file:data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_2040_sample_number_50.csv, data.shape (should be (2040, 50)): (2040, 50)
outcome_value:0 of all possible outcome_values:[0, 1]
outcome_value:1 of all possible outcome_values:[0, 1]
data file:data_no_sequence_10200_sample_number_50.csv, data.shape (should be (10200, 50)): (10200, 50)
data file:data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_10200_sample_number_50.csv, data.shape (should be (10200, 50)): (10200, 50)
outcome_value:0 of all possible outcome_values:[0, 1]
outcome_value:1 of all possible outcome_values:[0, 1]


In [6]:
# SANITY CHECK #1

# There should be 'sample' and 'outcome' columns
# 'sample' contains all phone numbers in period broken down into individual digits
# and combined
# 'outcome' is 0 or 1 for binary
# Also there should be mixed 0 and 1 since the samples have been randomize
df_random_2040_2040_combo_sample_number_50.tail()

,sample,outcome
419,"[3, 3, 0, 7, 8, 2, 6, 0, 3, 2, 8, 2, 3, 3, 1, ...",0
3844,"[5, 8, 1, 1, 7, 4, 7, 4, 2, 2, 6, 9, 9, 9, 6, ...",1
2814,"[6, 0, 7, 0, 5, 0, 8, 7, 4, 0, 2, 4, 0, 6, 9, ...",1
3523,"[8, 8, 9, 0, 5, 7, 5, 2, 7, 1, 9, 2, 1, 2, 0, ...",1
497,"[9, 7, 0, 1, 3, 0, 2, 2, 3, 1, 8, 5, 4, 9, 2, ...",0


In [7]:
# SANITY CHECK #2

# 50% or 2040 samples should be 1
np.count_nonzero(df_random_2040_2040_combo_sample_number_50['outcome'])

2040

In [8]:
# SANITY CHECK #3

# The 'sample' length should be same as SINGLE_SAMPLE_SIZE_50 = BATCH_NUMBER_COUNT_50 * NUMBER_SIZE
len(df_random_2040_2040_combo_sample_number_50['sample'][1])

750

In [9]:
# Split out dataframe containing both samples and outcome

(X_2040_2040_combo_sample_number_50_train, Y_2040_2040_combo_sample_number_50_train) = prepare_train_data(df_random_2040_2040_combo_sample_number_50, SINGLE_SAMPLE_SIZE_50)

(X_10200_10200_combo_sample_number_50_train, Y_10200_10200_combo_sample_number_50_train) = prepare_train_data(df_random_10200_10200_combo_sample_number_50, SINGLE_SAMPLE_SIZE_50)

type(X):<class 'list'>, len(X):4080, len(X[0]):750
df_row_count: 4080
single_sample_size_bit:750
X_train.shape:(4080, 1, 750, 1), Y_train.shape:(4080,)


type(Y[0]):<class 'numpy.int64'>, Y[0]:1
type(X):<class 'list'>, len(X):20400, len(X[0]):750
df_row_count: 20400
single_sample_size_bit:750
X_train.shape:(20400, 1, 750, 1), Y_train.shape:(20400,)


type(Y[0]):<class 'numpy.int64'>, Y[0]:1


In [10]:
# For CNN, the dimensions are number of samples, height, width, channel/feature maps
# Number of samples: sum of 'attack' and 'no attack' samples
# Height: 1
# Width: SINGLE_SAMPLE_SIZE_50
# Feature maps/Channels: 1
X_2040_2040_combo_sample_number_50_train.shape

(4080, 1, 750, 1)

In [11]:
Y_2040_2040_combo_sample_number_50_train.shape

(4080,)

In [12]:
# Various multi-class CNN models tested

def create_cnn_model_G_50():
  # Create model
  model = Sequential()
  # Max number of digits are different in number yet still can be within threshold
  digit_diff_size = 2
  digit_same_size = NUMBER_SIZE - digit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_digit_same_size = math.floor(digit_same_size/2)
  # add model layers
  number_digit_diff_size = math.floor(NUMBER_SIZE/digit_diff_size)
  model.add(Conv2D(250, kernel_size=(1,smallest_digit_same_size), strides=(1,1), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(Conv2D(50, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu'))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_coarse():
  # Create model
  model = Sequential()
  model.add(Conv2D(5, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(Conv2D(2, kernel_size=(1,1), strides=(1,1), activation='relu'))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_mod():
  # Create model
  model = Sequential()
  # Max number of bits are different in number yet still can be within threshold
  bit_diff_size = BIT_PER_DIGIT*2
  bit_same_size = NUMBER_SIZE_BIT - bit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_bit_same_size = int(math.floor(bit_same_size/2) / BIT_PER_DIGIT)
  # add model layers
  number_bit_diff_size = math.floor(NUMBER_SIZE_BIT/bit_diff_size)
  model.add(Conv2D(150, kernel_size=(1,smallest_bit_same_size), strides=(1,BIT_PER_DIGIT), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_BIT_50,1)))
  model.add(Conv2D(5, kernel_size=(1,NUMBER_SIZE_BIT ), strides=(1,NUMBER_SIZE_BIT), activation='relu'))
  model.add(Flatten())
  model.add(Dense(10 - THRESHOLD_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_maxpool():
  # Create model
  model = Sequential()
  # Max number of bits are different in number yet still can be within threshold
  # bit_diff_size = BIT_PER_DIGIT*2
  bit_same_size = NUMBER_SIZE_BIT - bit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_bit_same_size = math.floor(bit_same_size/2)
  # add model layers
  number_bit_diff_size = math.floor(NUMBER_SIZE_BIT/bit_diff_size)
  model.add(Conv2D(150, kernel_size=(1,smallest_bit_same_size), strides=(1,BIT_PER_DIGIT), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_BIT_50,1)))
  model.add(MaxPooling2D(pool_size=(1,smallest_bit_same_size), strides=(1,math.floor(smallest_bit_same_size/BIT_PER_DIGIT))))
  model.add(Conv2D(5, kernel_size=(1,1), strides=(1,1), activation='relu'))
  model.add(Flatten())
  model.add(Dense(10 - THRESHOLD_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_avepool():
  # Create model
  model = Sequential()
  # Max number of bits are different in number yet still can be within threshold
  digit_diff_size = 2
  digit_same_size = NUMBER_SIZE - digit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_digit_same_size = math.floor(digit_same_size/2)
  # add model layers
  number_digit_diff_size = math.floor(NUMBER_SIZE/digit_diff_size)
  model.add(Conv2D(150, kernel_size=(1,smallest_digit_same_size), strides=(1,1), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(AveragePooling2D(pool_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE)))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_coarse_3():
  # Create model
  model = Sequential()
  model.add(Conv2D(20, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  # MaxPooling2D 1x1 theoretically does nothing but without it the trained model is 5% less accurate
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  model.add(Conv2D(10, kernel_size=(1,1), strides=(1,1), activation='relu'))
  # MaxPooling2D 1x1 theoretically does nothing but without it the trained model is 5% less accurate
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [13]:
# For multi-class this is best so we use it here in binary class CNN: create_cnn_model_G_50_coarse_3

cnn_model_G_50_coarse_3 = create_cnn_model_G_50_coarse_3()

cnn_model_G_50_coarse_3.fit(X_10200_10200_combo_sample_number_50_train, Y_10200_10200_combo_sample_number_50_train, validation_split=0.2, batch_size=10, epochs=50)

Train on 16320 samples, validate on 4080 samples
Epoch 1/50
16320/16320 [==============================] - 9s - loss: 0.6935 - acc: 0.5041 - val_loss: 0.6882 - val_acc: 0.5311
Epoch 2/50
16320/16320 [==============================] - 9s - loss: 0.6732 - acc: 0.5694 - val_loss: 0.6595 - val_acc: 0.5831
Epoch 3/50
16320/16320 [==============================] - 9s - loss: 0.6456 - acc: 0.6118 - val_loss: 0.6382 - val_acc: 0.6316
Epoch 4/50
16320/16320 [==============================] - 9s - loss: 0.6171 - acc: 0.6475 - val_loss: 0.6253 - val_acc: 0.6409
Epoch 5/50
16320/16320 [==============================] - 9s - loss: 0.5897 - acc: 0.6784 - val_loss: 0.6333 - val_acc: 0.6400
Epoch 6/50
16320/16320 [==============================] - 10s - loss: 0.5667 - acc: 0.7023 - val_loss: 0.6446 - val_acc: 0.6292
Epoch 7/50
16320/16320 [==============================] - 10s - loss: 0.5385 - acc: 0.7238 - val_loss: 0.6550 - val_acc: 0.6314
Epoch 8/50
16320/16320 [==============================] - 10

In [14]:
predict_arr = cnn_model_G_50_coarse_3.predict_classes(X_2040_2040_combo_sample_number_50_train)
# Each element in 'a' will contain 1/True if prediction matches expected outcome
a = (predict_arr == np.array([[r] for r in Y_2040_2040_combo_sample_number_50_train]))
# % of correct predictions
np.count_nonzero(a)/np.size(a)

3904/4080 [===========================>..] - ETA: 0s

0.5977941176470588

In [15]:
# Alternatively you can evaluate using StratifiedKFold

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# Evaluate baseline model
seed = 7
estimator = KerasClassifier(build_fn=create_cnn_model_G_50_coarse_3, epochs=10, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_10200_10200_combo_sample_number_50_train, Y_10200_10200_combo_sample_number_50_train, cv=kfold)

print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
16320/16320 [==============================] - 15s - loss: 0.6895 - acc: 0.5224    
Epoch 2/10
16320/16320 [==============================] - 15s - loss: 0.6591 - acc: 0.5907    
Epoch 3/10
16320/16320 [==============================] - 14s - loss: 0.6373 - acc: 0.6225    
Epoch 4/10
16320/16320 [==============================] - 15s - loss: 0.6233 - acc: 0.6386    
Epoch 5/10
16320/16320 [==============================] - 15s - loss: 0.6054 - acc: 0.6587    
Epoch 6/10
16320/16320 [==============================] - 16s - loss: 0.5858 - acc: 0.6830    
Epoch 7/10
16320/16320 [==============================] - 15s - loss: 0.5599 - acc: 0.7045    
Epoch 8/10
16320/16320 [==============================] - 15s - loss: 0.5290 - acc: 0.7262    
Epoch 9/10
16320/16320 [==============================] - 17s - loss: 0.4905 - acc: 0.7558    
Epoch 10/10
16320/16320 [==============================] - 15s - loss: 0.6880 - acc: 0.5272    
Epoch 2/10
16320/16320 [=========================